# Initialize PyImageJ

In [1]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [2]:
Fiji_Local = r"C:\Users\confocal_microscope\Desktop\Tools\Fiji.app"

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.53t

ImageJ2 2.9.0/1.54b; Java 1.8.0_362 [amd64]; 191MB of 14542MB


In [3]:
def dump_info(image, CLI_print_title:str=None):
    """A handy function to print details of an image object."""
    name = image.name if hasattr(image, 'name') else None # xarray
    if name is None and hasattr(image, 'getName'): name = image.getName() # Dataset
    if name is None and hasattr(image, 'getTitle'): name = image.getTitle() # ImagePlus
    
    if CLI_print_title is not None: print(f"--> {CLI_print_title}:\n")
    print(f"    name  : {name or 'N/A'}")
    print(f"    type  : {type(image)}")
    print(f"    dtype : {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
    print(f"    shape : {image.shape if hasattr(image, 'shape') else 'N/A'}")
    print(f"    dims  : {image.dims if hasattr(image, 'dims') else 'N/A'}\n")

# Sub Functions

In [4]:
import logging


def init_logger(logger_name) -> logging.Logger:
    
    log: logging.Logger = logging.getLogger(logger_name)
    log.setLevel(logging.DEBUG)
    
    stream_handler: logging.StreamHandler = logging.StreamHandler()
    
    formatter: logging.Formatter = logging.Formatter('| %(asctime)s | %(name)s | %(levelname)s | %(message)s')
    stream_handler.setFormatter(formatter)
    log.addHandler(stream_handler)
    
    return log

In [5]:
import os
from tqdm.auto import tqdm


def create_new_dir(path:str, end="\n", display_in_CLI=True, use_tqdm=False):
    if not os.path.exists(path):
        # if the demo_folder directory is not exist then create it.
        os.makedirs(path)
        if use_tqdm: tqdm.write(f"path: '{path}' is created!{end}")
        elif display_in_CLI: print(f"path: '{path}' is created!{end}")

In [6]:
import re
from typing import *
import json


def check_len_of_name_list(name_list:List[str], target_len:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:
    dict_key = "len(image_name_list)"
    
    if len(name_list) != target_len:
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> len(image_name_list) = '{len(name_list)}', expect '{target_len}' " # WARNING:
    else: 
        check_dict[f"{dict_key:^24}"] = "PASS"
    
    return failed_cnt, check_dict


def check_Series_format(name_list:List[str], check_pos:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:
    
    # spelling of 'Series'
    dict_key = "spelling of 'Series'"
    temp_list = re.split("[0-9]", name_list[check_pos])
    if temp_list[0] != "Series":
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> '{temp_list[0]}', misspelling of 'Series' " # WARNING:
        check_dict[f"{'Series[num]':^24}"] = "NOT CHECK"
        return failed_cnt, check_dict
    else:
        check_dict[f"{dict_key:^24}"] = "PASS"
    
    # Series[num]
    dict_key = "Series[num]"
    try: 
        temp_list = name_list[check_pos].split("Series")
        num = int(temp_list[1])
        assert len(temp_list[1]) == 3, f"only {len(temp_list[1])} digits, expect 3 digits "
        check_dict[f"{dict_key:^24}"] = "PASS"
    except Exception as e: 
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> {e} " # WARNING:
    
    return failed_cnt, check_dict
    

def check_fish_format(name_list:List[str], check_pos:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:

    dict_key = "spelling of 'fish'"
    if name_list[check_pos] != "fish": 
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> '{name_list[check_pos]}', misspelling of 'fish' " # WARNING:
    else: 
        check_dict[f"{dict_key:^24}"] = "PASS"
        
    return failed_cnt, check_dict


def check_fishID_format(name_list:List[str], check_pos:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:
    
    dict_key = "fish_[ID]"
    try:
        int(name_list[check_pos])
        check_dict[f"{dict_key:^24}"] = "PASS"
    except Exception as e:
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> {e} " # WARNING:
    
    return failed_cnt, check_dict
    

def check_palmskin_format(name_list:List[str], check_pos:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:
    
    dict_key = "spelling of 'palmskin'"
    if name_list[check_pos] != "palmskin": 
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> '{name_list[check_pos]}', misspelling of 'palmskin' " # WARNING:
    else: 
        check_dict[f"{dict_key:^24}"] = "PASS"
    
    return failed_cnt, check_dict


def check_dpf_format(name_list:List[str], check_pos:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:

    # spelling of 'dpf'
    dict_key = "spelling of 'dpf'"
    temp_list = re.split("[0-9]", name_list[check_pos])
    if temp_list[-1] != "dpf":
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> '{temp_list[-1]}', misspelling of 'dpf' " # WARNING:
        check_dict[f"{'[num]_dpf':^24}"] = "NOT CHECK"
        return failed_cnt, check_dict
    else:
        check_dict[f"{dict_key:^24}"] = "PASS"
    
    # [num]_dpf
    dict_key = "[num]_dpf"
    try: 
        temp_list = name_list[check_pos].split("dpf")
        num = int(temp_list[0])
        check_dict[f"{dict_key:^24}"] = "PASS"
    except Exception as e: 
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> {e} " # WARNING:
    
    return failed_cnt, check_dict
    

def check_A_P_format(name_list:List[str], check_pos:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:
    
    dict_key = "A or P"
    if name_list[check_pos] != "A" and name_list[check_pos] != "P": 
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> '{name_list[check_pos]}', expect 'A' or 'P' " # WARNING:
    else: 
        check_dict[f"{dict_key:^24}"] = "PASS"

    return failed_cnt, check_dict


def check_RGB_format(name_list:List[str], check_pos:int, failed_cnt:int, check_dict:Dict[str, str]) -> Tuple[int, Dict[str, str]]:
    
    dict_key = "spelling of 'RGB'"
    if name_list[check_pos] != "RGB": 
        failed_cnt += 1
        check_dict[f"{dict_key:^24}"] = f"FAILED --> '{name_list[check_pos]}', misspelling of 'RGB' " # WARNING:
    else: 
        check_dict[f"{dict_key:^24}"] = "PASS"

    return failed_cnt, check_dict


# 20220610_CE001_palmskin_8dpf - Series001_fish_1_palmskin_8dpf_A (old format)
# 20221125_AI005_palmskin_10dpf - Series001_fish_165_A_RGB (new format)
def check_image_name(image_name:str, format_and_type:str) -> Tuple[int, Dict[str, str]]:

        split_list = re.split(" |_|-", str(image_name))
        # print(split_list)
        check_dict = {}
        check_dict[f"{'format_and_type':^24}"] = f"{format_and_type}"
        failed_cnt = 0
        
        if format_and_type == "old_rgb": failed_cnt, check_dict = check_len_of_name_list(split_list, 6, failed_cnt, check_dict)
        if format_and_type == "new_rgb": failed_cnt, check_dict = check_len_of_name_list(split_list, 5, failed_cnt, check_dict)
        if failed_cnt == 1: return failed_cnt, check_dict

        
        failed_cnt, check_dict = check_Series_format(split_list, 0, failed_cnt, check_dict)
        failed_cnt, check_dict = check_fish_format(split_list, 1, failed_cnt, check_dict)
        failed_cnt, check_dict = check_fishID_format(split_list, 2, failed_cnt, check_dict)
        
        if format_and_type == "old_rgb":
            failed_cnt, check_dict = check_palmskin_format(split_list, 3, failed_cnt, check_dict)
            failed_cnt, check_dict = check_dpf_format(split_list, 4, failed_cnt, check_dict)
            failed_cnt, check_dict = check_A_P_format(split_list, 5, failed_cnt, check_dict)
        
        if format_and_type == "new_rgb": 
            failed_cnt, check_dict = check_A_P_format(split_list, 3, failed_cnt, check_dict)
            failed_cnt, check_dict = check_RGB_format(split_list, 4, failed_cnt, check_dict)
        
        # print(failed_cnt)
        # print(json.dumps(check_dict, indent=2))

        return failed_cnt, check_dict

In [7]:
test_string = r"Series001_fish_1_palmskin_8dpf_A"
# test_string = r"Series001_fish_165_A_RG"


detect_old_new_list = re.findall("[RGB]", test_string)
# print(detect_old_new_list)
if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(test_string, "new_rgb")
else: failed_cnt, check_dict = check_image_name(test_string, "old_rgb")


test_string_list = re.split(" |_|-", test_string)
# while "" in test_string_list: test_string_list.remove("")
if (check_dict["    format_and_type     "] == "old_rgb") and (failed_cnt == 0):
    test_string_list.pop(3) # palmskin (old name only)
    test_string_list.pop(3) # [num]dpf (old name only)
    test_string_list.append("RGB")
image_name = "_".join(test_string_list)
print(image_name)


temp_str = ""
for key, value in check_dict.items(): 
    if value.split(" --> ")[0] == "FAILED" : temp_str += f"{key.strip()}, "
if failed_cnt > 0: print(f"FAILED : image_name, At least {failed_cnt} test failed: {temp_str}", end="")

Series001_fish_1_A_RGB


# Start Process

In [8]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# [IMPORTANT] RoiManager, ImageCalculator
from ij.plugin.frame import RoiManager
from ij.plugin import ImageCalculator
from ij.plugin import ChannelSplitter
rm = RoiManager()
imageCalculator = ImageCalculator()
channelSplitter = ChannelSplitter()


log = init_logger(r"{Test}_RGB_preprocess")

ImageJ Composite Functions

In [9]:
def median_R1_and_mean3D_R2(image):
    
    median_r1 = image.duplicate()
    ij.IJ.run(median_r1, "Median...", "radius=1 stack")
    
    median_r1_mean3D_r2 = median_r1.duplicate()
    ij.IJ.run(median_r1_mean3D_r2, "Mean 3D...", "x=2 y=2 z=2")
    
    ij.IJ.run(median_r1_mean3D_r2, "Z Project...", "projection=[Max Intensity]")
    median_r1_mean3D_r2_Zproj_max = ij.WindowManager.getCurrentImage()
    median_r1_mean3D_r2_Zproj_max.hide()
    
    return median_r1_mean3D_r2_Zproj_max


def group_show(*args):
    for image in args: image.show()

    
def group_hide(*args):
    for image in args: image.hide()

    
def crop_threshold_rect(image, roi):
    image.show()
    image.setRoi(roi)
    ij.IJ.run(image, "Fit Rectangle", "")
    cropped_img = image.crop()
    group_hide(image, cropped_img)
    return cropped_img

In [23]:
import re
from glob import glob
import numpy as np
from typing import TextIO


def single_RGB_preprocess(lif_path:str, total_lif_file:int, out_dir_root:str, log:logging.Logger, log_writer:TextIO):
    
    Reader.setId(lif_path)
    log.info(f'LIF_FILE : {lif_path}')
    
    seriesCount = Reader.getSeriesCount()
    log.info(f'seriesCount : {seriesCount}')
    
    for idx in range(seriesCount):
        
        ij.IJ.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{idx+1}")
        img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        # dump_info(img)
        # img.show()
        img.hide()
        
        
        # Get names and image dimensions
        
        # Consociate "palmskin_RGB_RAW" file_name
        file_name = lif_path.split(os.sep)[-1].split(".")[0]
        file_name_list = re.split(" |_|-", file_name)
        assert len(file_name_list) == 4, f"file_name format error, current : '{file_name}', expect like : '20221125_AI005_palmskin_10dpf.lif'"
        file_name = "_".join(file_name_list)
        
        # Get Prop("Image name")
        image_name = img.getProp("Image name")
        image_name = str(image_name)
        # Check format of 'image_name'
        detect_old_new_list = re.findall("[RGB]", image_name)
        if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(image_name, "new_rgb")
        else: failed_cnt, check_dict = check_image_name(image_name, "old_rgb")
        # Consociate image_name
        image_name_list = re.split(" |_|-", image_name)
        if (check_dict["    format_and_type     "] == "old_rgb") and (failed_cnt == 0):
            image_name_list.pop(3) # palmskin (old name only)
            image_name_list.pop(3) # [num]dpf (old name only)
            image_name_list.append("RGB")
        image_name = "_".join(image_name_list)
        
        # Combine 2 names above
        seN = f"{file_name} - {image_name}"
        img_dimensions = img.getDimensions()
        log.info(f"series {idx+1:{len(str(seriesCount))}}/{seriesCount} : '{seN}' , Dimensions : {img_dimensions} ( width, height, channels, slices, frames )")
        
        # Print FAILED of check 'image_name'
        temp_str = ""
        for key, value in check_dict.items(): 
            if value.split(" --> ")[0] == "FAILED" : temp_str += f"{key.strip()}, "
        if failed_cnt > 0: log.info(f"      #### ERROR : image_name, At least {failed_cnt} test failed: {temp_str}")
        
        # Write Log
        log_writer.write(f"|-- processing ...  series {idx+1:{len(str(seriesCount))}}/{seriesCount} in {idx+1}/{total_lif_file} \n")
        log_writer.write(f"|         {seN} \n")
        log_writer.write(f"|         Dimensions : {img_dimensions} ( width, height, channels, slices, frames ) \n")
        if failed_cnt > 0: log_writer.write(f"|      #### ERROR : image_name, At least {failed_cnt} test failed: {temp_str} \n")
        
        
        # Start image preprocessing
        
        # Create sub folder
        subfolder = os.path.join(out_dir_root, seN)
        dir_metaimg = os.path.join(subfolder, "MetaImage")
        create_new_dir(subfolder, display_in_CLI=False)
        create_new_dir(dir_metaimg, display_in_CLI=False)
        
        
        ij.IJ.run(img, "Set Scale...", "distance=2.2 known=1 unit=micron")
        # img.show()
        
        list = channelSplitter.split(img)
        Kuwahara_sampleing = 15
        
        
        ch_B = list[0]
        B_processed = median_R1_and_mean3D_R2(ch_B)
        ij.IJ.saveAsTiff(B_processed, os.path.normpath(f"{dir_metaimg}/01_B_processed.tif"))
        # Kuwahara Filter
        B_processed_Kuwahara = B_processed.duplicate()
        ij.IJ.run(B_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        ij.IJ.saveAsTiff(B_processed_Kuwahara, os.path.normpath(f"{dir_metaimg}/02_B_processed_Kuwahara{Kuwahara_sampleing}.tif"))
        
        
        ch_G = list[1]
        G_processed = median_R1_and_mean3D_R2(ch_G)
        ij.IJ.saveAsTiff(G_processed, os.path.normpath(f"{dir_metaimg}/03_G_processed.tif"))
        # Kuwahara Filter
        G_processed_Kuwahara = G_processed.duplicate()
        ij.IJ.run(G_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        ij.IJ.saveAsTiff(G_processed_Kuwahara, os.path.normpath(f"{dir_metaimg}/04_G_processed_Kuwahara{Kuwahara_sampleing}.tif"))
        
        
        ch_R = list[2]
        R_processed = median_R1_and_mean3D_R2(ch_R)
        ij.IJ.saveAsTiff(R_processed, os.path.normpath(f"{dir_metaimg}/05_R_processed.tif"))
        # Kuwahara Filter
        R_processed_Kuwahara = R_processed.duplicate()
        ij.IJ.run(R_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        ij.IJ.saveAsTiff(R_processed_Kuwahara, os.path.normpath(f"{dir_metaimg}/06_R_processed_Kuwahara{Kuwahara_sampleing}.tif"))
        
        
        # Merge, Enhance Contrast, without Kuwahara Filter
        group_show(R_processed, G_processed, B_processed) # needs to show image before merge
        ij.IJ.run("Merge Channels...", "c1=[05_R_processed.tif] c2=[03_G_processed.tif] c3=[01_B_processed.tif] create keep")
        stack_RGB = ij.WindowManager.getCurrentImage()
        group_hide(R_processed, G_processed, B_processed)
        ij.IJ.run(stack_RGB, "RGB Color", "") # will create new window
        composite_RGB_HE = ij.WindowManager.getCurrentImage()
        ij.IJ.run(composite_RGB_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        ij.IJ.saveAsTiff(composite_RGB_HE, os.path.normpath(f"{subfolder}/07_composite_RGB_HE.tif"))
        group_hide(stack_RGB, composite_RGB_HE)
        
        
        # Merge, Enhance Contrast, with Kuwahara Filter
        group_show(R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara) # needs to show image before merge
        ij.IJ.run("Merge Channels...", f"c1=[06_R_processed_Kuwahara{Kuwahara_sampleing}.tif] c2=[04_G_processed_Kuwahara{Kuwahara_sampleing}.tif] c3=[02_B_processed_Kuwahara{Kuwahara_sampleing}.tif] create keep")
        stack_RGB_Kuwahara = ij.WindowManager.getCurrentImage()
        group_hide(R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara)
        ij.IJ.run(stack_RGB_Kuwahara, "RGB Color", "") # will create new window
        composite_RGB_Kuwahara_HE = ij.WindowManager.getCurrentImage()
        ij.IJ.run(composite_RGB_Kuwahara_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        ij.IJ.saveAsTiff(composite_RGB_Kuwahara_HE, os.path.normpath(f"{subfolder}/08_composite_RGB_Kuwahara_HE.tif"))
        group_hide(stack_RGB_Kuwahara, composite_RGB_Kuwahara_HE)
        
        
        # Average with and without Kuwahara Filter
        composite_HE_mix = imageCalculator.run(composite_RGB_HE, composite_RGB_Kuwahara_HE, "Average create")
        ij.IJ.saveAsTiff(composite_HE_mix, os.path.normpath(f"{subfolder}/09_composite_HE_mix.tif"))
        
        
        # Generate Mask using Bright Field
        ch_BF = list[3]
        ij.IJ.run(ch_BF, "Z Project...", "projection=Median")
        BF_Zproj_median = ij.WindowManager.getCurrentImage()
        ij.IJ.run(BF_Zproj_median, "Enhance Contrast...", "saturated=0.35 equalize")
        ij.IJ.run(BF_Zproj_median, "Subtract Background...", "rolling=1000 create")
        ij.IJ.run(BF_Zproj_median, "Auto Threshold", "method=Huang")
        ij.IJ.run("Set Measurements...", "area feret's display redirect=None decimal=2")
        ij.IJ.run(BF_Zproj_median, "Analyze Particles...", "size=0-Infinity show=Masks display include add")
        mask_img = ij.WindowManager.getCurrentImage()
        ij.IJ.run(mask_img, "Invert", "")
        ij.IJ.saveAsTiff(mask_img, os.path.normpath(f"{dir_metaimg}/10_ROI_Mask.tif"))
        group_hide(BF_Zproj_median, mask_img)
        
        
        # Check ROI
        rm.runCommand("Show All with labels")
        rm_size = int(rm.getCount())
        if rm_size == 1:
            rm.save(os.path.normpath(f"{dir_metaimg}/RoiSet.zip"))
        else:
            log.info(f'      ROI in RoiManager: {rm_size}')
            log.info(f'      #### ERROR : Number of ROI not = 1')
            # Write Log
            log_writer.write(f"|         number of ROI = {rm_size} \n")
            log_writer.write("| #### ERROR : Number of ROI not = 1 \n") # ERROR:
        
        
        # Fit ROI with Rect 
        mask_rect = mask_img.duplicate()
        ij.IJ.setForegroundColor(255, 255, 255)
        ij.IJ.setBackgroundColor(0, 0, 0)
        mask_rect.show()
        mask_rect.setRoi(rm.getRoi(0))
        ij.IJ.run(mask_rect, "Fit Rectangle", "")
        ij.IJ.run(mask_rect, "Fill", "slice")
        mask_rect.hide()
        
        
        # Average with Mask_Rect
        composite_RGB_HE_AvgMaskRect = imageCalculator.run(composite_RGB_HE, mask_rect, "Average create")
        composite_RGB_Kuwahara_HE_AvgMaskRect = imageCalculator.run(composite_RGB_Kuwahara_HE, mask_rect, "Average create")
        composite_HE_mix_AvgMaskRect = imageCalculator.run(composite_HE_mix, mask_rect, "Average create")
        ij.IJ.saveAsTiff(composite_RGB_HE_AvgMaskRect, os.path.normpath(f"{dir_metaimg}/11_composite_RGB_HE--MIX.tif"))
        ij.IJ.saveAsTiff(composite_RGB_Kuwahara_HE_AvgMaskRect, os.path.normpath(f"{dir_metaimg}/12_composite_RGB_Kuwahara_HE--MIX.tif"))
        ij.IJ.saveAsTiff(composite_HE_mix_AvgMaskRect, os.path.normpath(f"{dir_metaimg}/13_composite_HE_mix--MIX.tif"))
        
        
        # crop_threshold_roi, save_as_tiff
        cropped_composite_RGB_HE = crop_threshold_rect(composite_RGB_HE, rm.getRoi(0))
        cropped_composite_RGB_Kuwahara_HE = crop_threshold_rect(composite_RGB_Kuwahara_HE, rm.getRoi(0))
        cropped_composite_HE_mix = crop_threshold_rect(composite_HE_mix, rm.getRoi(0))
        ij.IJ.saveAsTiff(cropped_composite_RGB_HE, os.path.normpath(f"{subfolder}/14_cropped_composite_RGB_HE.tif"))
        ij.IJ.saveAsTiff(cropped_composite_RGB_Kuwahara_HE, os.path.normpath(f"{subfolder}/15_cropped_composite_RGB_Kuwahara_HE.tif"))
        ij.IJ.saveAsTiff(cropped_composite_HE_mix, os.path.normpath(f"{subfolder}/16_cropped_composite_HE_mix.tif"))
        
        
        # Show image
        composite_HE_mix_AvgMaskRect.show()
        composite_HE_mix_AvgMaskRect.hide()
        
        rm.runCommand("Deselect")
        rm.runCommand("Delete")
        ij.IJ.run("Clear Results", "")
        ij.IJ.run("Close All", "")
        log_writer.write(f"| \n") # make Log file looks better.

In [26]:
ij.IJ.run("Close All", "")

In [27]:
# File Input
rgb_raw_lif_source = r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\palmskin_RGB_RAW"

# Result Output
ap_data = r"C:\Users\confocal_microscope\Desktop\{PyIJ_OutTest}_RGB_preprocess" # r"C:\Users\confocal_microscope\Desktop\WorkingDir\(D2)_Image_AP\{Data}_Data\{20221209_UPDATE_82}_Academia_Sinica_i324"
preprocess_method_desc = "ch4_median_proj, thres_Huang"
preprocess_root = os.path.join(ap_data, f"{{{preprocess_method_desc}}}_RGB_preprocess")
create_new_dir(preprocess_root, display_in_CLI=False)


# Scan Leica LIF file
lif_path_list  = glob(os.path.normpath(f"{rgb_raw_lif_source}/*/*.lif"))
log.info(f'lif_path_list {type(lif_path_list)}: {lif_path_list}, [ found {len(lif_path_list)} files ] \n')


# Create log writer
log_path = os.path.join(preprocess_root, "Logs.log")
log_writer = open(log_path, mode="w")
log.info(f'log_writer {type(log_writer)}: {log_writer}')


# {test} single RGB raw lif
single_lif_path =  r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\palmskin_RGB_RAW\Before_20221109\20220610 CE001 palmskin_8dpf.lif"


for i, lif in enumerate(lif_path_list): 
    
    # Write Log
    log_writer.write("\n\n\n")
    log_writer.write(f"|-----------------------------------------  Processing ... {i+1}/{len(lif_path_list)}  ----------------------------------------- \n")
    log_writer.write(f"| \n")
    log_writer.write(f"|         LIF_FILE : {lif.split(os.sep)[-1]} \n")
    log_writer.write(f"| \n")
    
    # process current LIF_FILE
    single_RGB_preprocess(lif, len(lif_path_list), preprocess_root, log, log_writer)
    log.info("\n") # make CLI output looks better.


log_writer.close()
rm.close()
ij.WindowManager.closeAllWindows()

log.info(" -- finished --")

| 2023-02-23 22:58:14,063 | {Test}_RGB_preprocess | INFO | lif_path_list <class 'list'>: ['C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220610 CE001 palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220613 CE001 palmskin_11dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220617 CE002 palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220620 CE002 palmskin_11dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin

In [25]:
log_writer.close()